In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.data import astronaut, checkerboard
from skimage.feature import corner_harris, corner_peaks
from skimage.color import rgb2gray


# My Harris Corner Detector

In [ ]:
#Loading the images from skimage
astronaut_image = astronaut()
checkerboard_image = checkerboard()

In [3]:
# Sobel x-axis kernel
SOBEL_X = np.array((
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]), dtype="int32")

# Sobel y-axis kernel
SOBEL_Y = np.array((
    [-1, -2, -1],
    [0, 0, 0],
    [1, 2, 1]), dtype="int32")

# Gaussian kernel
GAUSS = np.array((
    [1/16, 2/16, 1/16],
    [2/16, 4/16, 2/16],
    [1/16, 2/16, 1/16]), dtype="float64")


In [4]:
def convolve(image, kernel):
    # Checking if the kernel dimensions are odd
    if kernel.shape[0] % 2 != 1 or kernel.shape[1] % 2 != 1:
        raise ValueError("Only odd dimensions on filter supported")

    # Get the height and width of the input image
    image_height = image.shape[0]
    image_width = image.shape[1]

    # Calculate the padding required for convolution
    pad_height = kernel.shape[0] // 2
    pad_width = kernel.shape[1] // 2

    # Create a padding configuration for the image
    padding = ((pad_height, pad_height), (pad_width, pad_width))

    # Create an empty result image (output of convolution)
    convolved_image = np.empty(image.shape, dtype=np.float64)

    # Pad the input image with zeros
    image = np.pad(image, padding, mode='constant', constant_values=0)

    # Perform convolution
    for i in np.arange(pad_height, image_height + pad_height):
        for j in np.arange(pad_width, image_width + pad_width):
            # Extract the region of interest (ROI)
            roi = image[i - pad_height:i + pad_height + 1, j - pad_width:j + pad_width + 1]

            # Calculate the convolution result for the current pixel
            convolved_image[i - pad_height, j - pad_width] = (roi * kernel).sum()

    # Normalize the result based on data type
    if (convolved_image.dtype == np.float64):
        # Normalize and convert the kernel for float results
        kernel = kernel / 255.0
        kernel = (kernel * 255).astype(np.uint8)
    else:
        # Normalize the result for other data types
        convolved_image = convolved_image + abs(np.amin(convolved_image))
        convolved_image = convolved_image / np.amax(convolved_image)
        convolved_image = (convolved_image * 255.0)

    return convolved_image


In [ ]:
def my_harris_corner_detector(image, threshold=0.06, window_size=5, k=0.05):
    # Convert the image to grayscale if it has multiple channels
    gray_image = image if len(image.shape) == 2 else rgb2gray(image)

    # Define Sobel filters for gradient computation
    sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

    # Compute gradients using Sobel filters
    I_x = convolve(gray_image, sobel_x)
    I_y = convolve(gray_image, sobel_y)

    # Calculate elements of the Harris matrix
    I_x2 = I_x**2
    I_y2 = I_y**2
    I_xy = I_x * I_y

    # Compute sums of squared gradients within the specified window
    kernel = np.ones((window_size, window_size))
    S_x2 = convolve(I_x2, kernel)
    S_y2 = convolve(I_y2, kernel)
    S_xy = convolve(I_xy, kernel)

    # Calculate the Harris response
    det = S_x2 * S_y2 - S_xy**2
    trace = S_x2 + S_y2
    harris_response = det - k * (trace**2)

    # Apply thresholding to the Harris response
    corners = np.where(harris_response > threshold)

    return corners


In [ ]:
def visualize_original(image, title):
    # Display the astronaut image
    plt.imshow(image)
    plt.title(title)
    plt.imshow(image, cmap='gray')
    plt.show()

def visualize_corners(image, title, corners_my, corners_skimage):
    plt.imshow(image, cmap='gray')
    plt.title(title)
    plt.plot(corners_my[1], corners_my[0], 'ro', markersize=3, label='My Corners')
    plt.plot(corners_skimage[:, 1], corners_skimage[:, 0], 'b+', markersize=5, label='Skimage Corners')
    plt.legend()
    plt.show()


def main():
    # Load the input images
    astronaut_image = astronaut()
    checkerboard_image = checkerboard()
    
    # Visualize the original images
    visualize_original(astronaut_image, 'Astronaut Original Image')
    visualize_original(checkerboard_image, 'Checkerboard Original Image')
    
    # Detect corners using my_harris_corner_detector1
    corners_astronaut = my_harris_corner_detector(astronaut_image, threshold= 0.5,window_size=7, k=0.1)
    corners_checkerboard = my_harris_corner_detector(checkerboard_image, threshold= 0.5,window_size=6, k=0.01)

    # Detect corners using Skimage's corner_harris and corner_peaks
    harris_response_astronaut = corner_harris(rgb2gray(astronaut_image))
    corners_peaks_astronaut = corner_peaks(harris_response_astronaut)
    
    harris_response_checkerboard = corner_harris(checkerboard_image)
    corners_peaks_checkerboard = corner_peaks(harris_response_checkerboard)
    

    #Visualize the corner images
    visualize_corners(astronaut_image, 'Astronaut Cornered Image', corners_astronaut, corners_peaks_astronaut)
    visualize_corners(checkerboard_image, 'Checkerboard Cornered Image', corners_checkerboard, corners_peaks_checkerboard)

if __name__ == "__main__":
    main()
